In [1]:
import numpy as np
import os
import pandas as pd
import copy 
import datetime 
data_path = '../../../code_data/shanghai_t1dm/'
process_file = 'nips_23'

# pid_list = [1000 + d for d in range(1, 13)]


In [2]:
names = ['ts', 'glucose_level', 'finger_stick', 'blood_ketone', 'meal', 'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',	'insulin_iv',] 
usecols = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]
pid2df = {}
for x in os.listdir(os.path.join(data_path,
                                  'processing/temp1')):
    if x.endswith('.xlsx'):
        # Prints only text file present in My Folder
        print(x)
        pid = int(x[:4])
        df = pd.read_excel(os.path.join(data_path, 'processing/temp1',x), names=names, usecols=usecols)
        if pid not in pid2df:
            pid2df[pid] = df
        else:
            pid2df[pid] = pd.concat([pid2df[pid], df])

1001_0_20210730.xlsx
1002_0_20210504.xlsx
1002_1_20210521.xlsx
1002_2_20210909.xlsx
1003_0_20210831.xlsx
1004_0_20210425.xlsx
1005_0_20210522.xlsx
1006_0_20210114.xlsx
1006_1_20210209.xlsx
1006_2_20210303.xlsx
1007_0_20210726.xlsx
1008_0_20210713.xlsx
1009_0_20210803.xlsx
1010_0_20210915.xlsx
1011_0_20210622.xlsx
1012_0_20210923.xlsx


In [3]:
import re
def change_meal(x):
    if pd.notna(x):
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan
        return np.array(result, dtype=float).sum()
for pid in pid2df:
    pid2df[pid]['meal'] = pid2df[pid]['meal'].apply(change_meal)

In [4]:
for pid in pid2df:
    pid2df[pid]['meal'] = pid2df[pid]['meal'].astype(np.float64)

In [5]:
def change_insulin(x):
    if pd.notna(x):
        if type(x) != str:
            return x
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+\s*IU')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan

        result = [res[:-3] for res in result]
        final = np.array(result, dtype=float).sum()
        return final
for pid in pid2df:
    pid2df[pid]['insulin_dose_sc'] = pid2df[pid]['insulin_dose_sc'].apply(change_insulin)
    # pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].apply(change_insulin)
    pid2df[pid]['insulin_bolus'] = pid2df[pid]['insulin_bolus'].apply(change_insulin)
    pid2df[pid]['basal'] = pid2df[pid]['basal'].apply(change_insulin)
    pid2df[pid]['insulin_iv'] = pid2df[pid]['insulin_iv'].apply(change_insulin)

In [6]:
def nan_sum(x):
    if pd.notna(x[0]) and pd.notna(x[1]):
        return x[0] + x[1]
    elif pd.notna(x[0]):
        return x[0]
    elif pd.notna(x[1]):
        return x[1]
    else:
        return np.NaN

for pid in pid2df:
    pid2df[pid]['insulin_dose_sc'] = pid2df[pid]['insulin_dose_sc'].astype(np.float64)
    pid2df[pid]['insulin_bolus'] = pid2df[pid]['insulin_bolus'].astype(np.float64)
    pid2df[pid]['basal'] = pid2df[pid]['basal'].astype(np.float64)
    pid2df[pid]['insulin_iv'] = pid2df[pid]['insulin_iv'].astype(np.float64)
    pid2df[pid]['bolus'] = pid2df[pid][['insulin_dose_sc', 'insulin_bolus']].apply(nan_sum, axis=1)

In [7]:
def change_non_insulin(x):
    if pd.notna(x):
        if type(x) != str:
            return x
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan
        final = np.array(result, dtype=float).sum()
        return final
for pid in pid2df:
    pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].apply(change_non_insulin)


In [8]:
for pid in pid2df:
    pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].astype(np.float64)

In [9]:
pid2regular_df = {}
pid_list = list(pid2df.keys())
for pid in pid_list:
    df = pid2df[pid]
    df = df.set_index('ts')
    df = df.sort_index()
    pid2regular_df[pid] = df
    print(f'{pid}, total_points:{len(df)}, start:{df.index[0]}, end:{df.index[-1]}')


1001, total_points:658, start:2021-07-30 16:43:00, end:2021-08-06 12:58:00
1002, total_points:2238, start:2021-05-04 10:33:00, end:2021-09-13 09:06:00
1003, total_points:1339, start:2021-08-31 17:43:00, end:2021-09-14 16:13:00
1004, total_points:917, start:2021-04-25 17:26:00, end:2021-05-05 06:26:00
1005, total_points:1256, start:2021-05-22 11:59:00, end:2021-06-04 13:44:00
1006, total_points:3710, start:2021-01-14 18:39:00, end:2021-03-15 10:38:00
1007, total_points:1337, start:2021-07-26 09:58:00, end:2021-08-09 07:58:00
1008, total_points:766, start:2021-07-13 16:27:00, end:2021-07-21 15:42:00
1009, total_points:681, start:2021-08-03 12:04:00, end:2021-08-10 14:04:00
1010, total_points:918, start:2021-09-14 17:30:00, end:2021-09-24 06:45:00
1011, total_points:536, start:2021-06-22 17:23:00, end:2021-06-28 07:08:00
1012, total_points:1339, start:2021-09-23 01:08:00, end:2021-10-06 23:38:00


In [11]:
# add time features

def add_time_attributes(pid2data):
    for pid in pid2data:
        data = pid2data[pid]

        temp = data.index.to_frame().iloc[:, 0].dt
        
        data['day_of_week'] = temp.dayofweek
        data['day_of_week'] = data['day_of_week'].astype(np.float64)

        data['hour'] = temp.hour
        data['hour'] = data['hour'].astype(np.float64)

        data['minute'] = temp.minute
        data['minute'] = data['minute'].astype(np.float64)

        
        data['timestamp'] = temp.hour * 3600 +\
                                temp.minute * 60 +\
                                temp.second
        data['timestamp'] = data['timestamp'].astype(np.float64)
        
        # new ————————————————————————
        seconds_in_day = 24*60*60

        data['sin_time'] = np.sin(2 * np.pi * data.timestamp / seconds_in_day)
        data['cos_time'] = np.cos(2 * np.pi * data.timestamp / seconds_in_day)
        data['sin_time'].astype(np.float64)
        data['cos_time'].astype(np.float64)
        # end ______________________
        
        data['datastamp'] = temp.year * 10000 + temp.month * 100 + temp.day
        
        
    return pid2data
pid2regular_df = add_time_attributes(pid2regular_df)
for pid in pid2regular_df:
    data = pid2regular_df[pid]
    data = data.loc[pd.notna(data['glucose_level'])]
    pid2regular_df[pid] = data

In [12]:
def f(x):
    return 1.509 * ((np.log(x))**1.084 - 5.381)

def cal_LBGI(x):
    x = f(x) if f(x) <= 0 else 0
    return 10 * x ** 2

def cal_HBGI(x):
    x = f(x) if f(x) > 0 else 0
    return 10 * x ** 2
count2count_list = {}
count2count_list['No. of Days'] = []
count2count_list['CV'] = []
count2count_list['TIR'] = []
count2count_list['TBR'] = []
count2count_list['TAR'] = []

count2count_list['LBGI'] = []
count2count_list['HBGI'] = []

count2count_list['Mean of CGM data'] = []
count2count_list['SD of CGM data'] = []
count2count_list['No. of CGM records'] = []

for pid in pid2regular_df:
    
    glucose_avg_by_day = pid2regular_df[pid][['glucose_level', 'datastamp']].groupby('datastamp').mean()
    
    count2count_list['No. of Days'].append(len(glucose_avg_by_day))
    
    glucose_mean = pid2regular_df[pid]['glucose_level'].mean()
    glucose_std = pid2regular_df[pid]['glucose_level'].std()
    count2count_list['Mean of CGM data'].append(glucose_mean)
    count2count_list['SD of CGM data'].append(glucose_std)
    
    count2count_list['No. of CGM records'].append(len(pid2regular_df[pid]['glucose_level']))
 
    count2count_list['CV'].append((glucose_std / glucose_mean) * 100)

    target_range_min = 70
    target_range_max = 180
    
    glucose_values = pid2regular_df[pid]['glucose_level']
    time_in_range = ((glucose_values >= target_range_min) & (glucose_values <= target_range_max)).sum()
    total_time_points = len(glucose_values)
    tir_percentage = (time_in_range / total_time_points) * 100
    count2count_list['TIR'].append(tir_percentage)
    
    
    time_below_range = ((glucose_values < target_range_min)).sum()
    tbr_percentage = (time_below_range / total_time_points) * 100
    count2count_list['TBR'].append(tbr_percentage)
    
    time_above_range = ((glucose_values > target_range_max)).sum()
    tar_percentage = (time_above_range / total_time_points) * 100
    count2count_list['TAR'].append(tar_percentage)
    
    LBGI = pid2regular_df[pid]['glucose_level'].apply(cal_LBGI)
    HBGI = pid2regular_df[pid]['glucose_level'].apply(cal_HBGI)

    count2count_list['LBGI'].append(LBGI.mean())
    count2count_list['HBGI'].append(HBGI.mean())
    


In [14]:
import xlwt

book = xlwt.Workbook(encoding='utf-8', style_compression=0)
sheet = book.add_sheet('Main', cell_overwrite_ok=True)

row_names = ['No. of Days', 'No. of CGM records', 'Mean of CGM data', 'SD of CGM data', 'TIR', 'TBR','TAR', 'CV', 'LBGI', 'HBGI'] 


for r_idx, r_name in enumerate(row_names):
    sheet.write(r_idx + 3, 0, r_name)
    
    if 'No' not in r_name:
        print_str = f'{np.mean(count2count_list[r_name]):.2f}({np.std(count2count_list[r_name]):.2f})'
    else:
        print_str = f'{int(np.mean(count2count_list[r_name]))}({int(np.std(count2count_list[r_name]))})'
    sheet.write(r_idx + 3, 1, print_str)
book.save(f'shanghai_t1dm.xls')
for k in count2count_list:

   print(k, np.mean(count2count_list[k]), np.std(count2count_list[k]))

No. of Days 15.083333333333334 9.595644266483042
CV 38.30259615774941 7.163053893444234
TIR 53.84196540777322 12.258566907988577
TBR 6.645055895167601 6.4537399921136345
TAR 39.51297869705917 16.40448268023312
LBGI 1.6324020123418546 1.5498868453417116
HBGI 8.87287509510201 3.5479064678307997
Mean of CGM data 166.51329635718534 27.81495467857983
SD of CGM data 62.75056268312537 11.948562292428583
No. of CGM records 1307.9166666666667 849.0351836382022
